In [1]:
import pandas as pd
# import pyodbc
import logging
import requests
import copy
import math
import time
import nltk
from elasticsearch import helpers

from elasticsearch import Elasticsearch


In [7]:
df_materiais = pd.read_csv('df_materiais.csv')
df_materiais

,Unnamed: 0,ID_MATERIAL,TIPO,CODIGO_MATERIAL,ITEM_SUSPENSO,MATERIAL_SUSTENTAVEL,CODIGO_CLASSE,DESCRICAO_CLASSE,CODIGO_SUBCLASSE,DESCRICAO_SUBCLASSE,CODIGO_GRUPO,DESCRICAO_GRUPO,CODIGO_DIVISAO,DESCRICAO_DIVISAO,CODIGO_SECAO,DESCRICAO_SECAO,CARACTERISTICAS_BUSCA,NOME_MATERIAL,MATERIAL_TEXT
0,0,431550,M,600581,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Resistência Nominal 1k8OHMS Potência Nominal 1...,Resistor Carbono,Resistor Carbono Resistência Nominal 1k8OHMS P...
1,1,431551,M,600580,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
2,2,431552,M,600579,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
3,3,431553,M,600578,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
4,4,431554,M,600576,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Tipo Fixo Aplicação Eletrônica Resistência Nom...,Resistor Carbono,Resistor Carbono Tipo Fixo Aplicação Eletrônic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253861,253861,683446,M,377520,S,N,6145,FIOS E CABOS ELÉTRICOS,NaN,NaN,61,CONDUTORES ELÉTRICOS E EQUIPAMENTOS PARA GERAÇ...,NaN,NaN,NaN,NaN,Aplicação Uso Em Vídeo Características Adicion...,Cabo De Vídeo,Cabo De Vídeo Aplicação Uso Em Vídeo Caracterí...
253862,253862,683447,M,377519,S,N,6145,FIOS E CABOS ELÉTRICOS,NaN,NaN,61,CONDUTORES ELÉTRICOS E EQUIPAMENTOS PARA GERAÇ...,NaN,NaN,NaN,NaN,Aplicação Uso Em Vídeo N° De Cabos 3UN Tipo An...,Cabo De Vídeo,Cabo De Vídeo Aplicação Uso Em Vídeo N° De Cab...
253863,253863,683780,M,612103,N,N,6540,"EQUIPAMENTOS, INSTRUMENTOS E SUPRIMENTOS OFTAL...",NaN,NaN,65,"EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁR...",NaN,NaN,NaN,NaN,Tipo Grande Ocular Modelo De Não Contato Aplic...,Sistema De Lentes Para Equipamento Oftalmológico,Sistema De Lentes Para Equipamento Oftalmológi...
253864,253864,683781,M,229763,N,N,6540,"EQUIPAMENTOS, INSTRUMENTOS E SUPRIMENTOS OFTAL...",NaN,NaN,65,"EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁR...",NaN,NaN,NaN,NaN,Material Cristal Características Adicionais Au...,Sistema De Lentes Para Equipamento Oftalmológico,Sistema De Lentes Para Equipamento Oftalmológi...


In [2]:
df_index_catmat = pd.read_csv('df_index_catmat_resample.csv')
df_index_catmat

/tmp/ipykernel_1326497/1942492608.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_index_catmat = pd.read_csv('df_index_catmat_resample.csv')


,Unnamed: 0.1,Unnamed: 0,ID_MATERIAL,TIPO,CODIGO_MATERIAL,ITEM_SUSPENSO,MATERIAL_SUSTENTAVEL,CODIGO_CLASSE,DESCRICAO_CLASSE,CODIGO_SUBCLASSE,DESCRICAO_SUBCLASSE,CODIGO_GRUPO,DESCRICAO_GRUPO,CODIGO_DIVISAO,DESCRICAO_DIVISAO,CODIGO_SECAO,DESCRICAO_SECAO,CARACTERISTICAS_BUSCA,NOME_MATERIAL,MATERIAL_TEXT
0,0,0.0,431550,M,600581.0,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Resistência Nominal 1k8OHMS Potência Nominal 1...,Resistor Carbono,Resistor Carbono Resistência Nominal 1k8OHMS P...
1,1,1.0,431551,M,600580.0,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
2,2,2.0,431552,M,600579.0,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
3,3,3.0,431553,M,600578.0,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
4,4,4.0,431554,M,600576.0,N,N,5905,RESISTORES,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Tipo Fixo Aplicação Eletrônica Resistência Nom...,Resistor Carbono,Resistor Carbono Tipo Fixo Aplicação Eletrônic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244333,253811,253861.0,683446,M,377520.0,S,N,6145,FIOS E CABOS ELÉTRICOS,NaN,NaN,61,CONDUTORES ELÉTRICOS E EQUIPAMENTOS PARA GERAÇ...,NaN,NaN,NaN,NaN,Aplicação Uso Em Vídeo Características Adicion...,Cabo De Vídeo,Cabo De Vídeo Aplicação Uso Em Vídeo Caracterí...
244334,253812,253862.0,683447,M,377519.0,S,N,6145,FIOS E CABOS ELÉTRICOS,NaN,NaN,61,CONDUTORES ELÉTRICOS E EQUIPAMENTOS PARA GERAÇ...,NaN,NaN,NaN,NaN,Aplicação Uso Em Vídeo N° De Cabos 3UN Tipo An...,Cabo De Vídeo,Cabo De Vídeo Aplicação Uso Em Vídeo N° De Cab...
244335,253813,253863.0,683780,M,612103.0,N,N,6540,"EQUIPAMENTOS, INSTRUMENTOS E SUPRIMENTOS OFTAL...",NaN,NaN,65,"EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁR...",NaN,NaN,NaN,NaN,Tipo Grande Ocular Modelo De Não Contato Aplic...,Sistema De Lentes Para Equipamento Oftalmológico,Sistema De Lentes Para Equipamento Oftalmológi...
244336,253814,253864.0,683781,M,229763.0,N,N,6540,"EQUIPAMENTOS, INSTRUMENTOS E SUPRIMENTOS OFTAL...",NaN,NaN,65,"EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁR...",NaN,NaN,NaN,NaN,Material Cristal Características Adicionais Au...,Sistema De Lentes Para Equipamento Oftalmológico,Sistema De Lentes Para Equipamento Oftalmológi...


In [8]:
filtered_df = df_materiais[df_materiais['MATERIAL_TEXT'].str.split().apply(len) > 5]
filtered_df
df_catmar_relv = filtered_df.sample(40)
df_catmar_relv

,Unnamed: 0,ID_MATERIAL,TIPO,CODIGO_MATERIAL,ITEM_SUSPENSO,MATERIAL_SUSTENTAVEL,CODIGO_CLASSE,DESCRICAO_CLASSE,CODIGO_SUBCLASSE,DESCRICAO_SUBCLASSE,CODIGO_GRUPO,DESCRICAO_GRUPO,CODIGO_DIVISAO,DESCRICAO_DIVISAO,CODIGO_SECAO,DESCRICAO_SECAO,CARACTERISTICAS_BUSCA,NOME_MATERIAL,MATERIAL_TEXT
243461,243461,671843,M,336123,N,N,6550,"SUBSTÂNCIAS PARA DIAGNÓSTICO ""IN VITRO"", REAG...",NaN,NaN,65,"EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁR...",NaN,NaN,NaN,NaN,Tipo Monoclonal De Camundongo Atividade Anti C...,Anticorpo,Anticorpo Tipo Monoclonal De Camundongo Ativid...
93005,93005,523569,M,602982,N,N,5120,"FERRAMENTAS MANUAIS SEM CORTE, NÃO ACIONADAS ...",NaN,NaN,51,FERRAMENTAS MANUAIS,NaN,NaN,NaN,NaN,Padrão Furo Quadrado Passo 2x1 Tipo Pente Perf...,Ferramenta,Ferramenta Padrão Furo Quadrado Passo 2x1 Tipo...
188469,188469,617401,M,353799,N,N,8305,TECIDOS,NaN,NaN,83,"TECIDOS, COUROS, PELES, AVIAMENTOS, BARRACAS E...",NaN,NaN,NaN,NaN,Cor Preta Largura 8MB Aplicação Cobertura E Pr...,Lona Plástica,Lona Plástica Cor Preta Largura 8MB Aplicação ...
14536,14536,446050,M,258591,S,N,7020,IMPRESSORAS,NaN,NaN,70,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS...",NaN,NaN,NaN,NaN,Tensão Alimentação 110/220V Resolução Impressã...,Impressora Laser,Impressora Laser Tensão Alimentação 110/220V R...
173541,173541,602760,M,448817,S,N,4120,EQUIPAMENTO DE AR CONDICIONADO,NaN,NaN,41,"EQUIPAMENTOS PARA REFRIGERAÇÃO, AR CONDICIONAD...",NaN,NaN,NaN,NaN,Tipo Split Capacidade Refrigeração 24.000KGF M...,Aparelho Ar Condicionado,Aparelho Ar Condicionado Tipo Split Capacidade...
209791,209791,638470,M,336475,N,N,5975,FERRAGENS E SUPRIMENTOS DE ELETRICIDADE,NaN,NaN,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,NaN,NaN,NaN,NaN,Material Pvc Cor Cinza Bitola 3/4POL Aplicação...,Tampa Condulete,Tampa Condulete Material Pvc Cor Cinza Bitola ...
182921,182921,611887,M,228623,S,N,7070,PEÇAS E ACESSÓRIOS PARA IMPRESSORAS,NaN,NaN,70,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS...",NaN,NaN,NaN,NaN,Cor Amarela Referência Impressora Tektronix 56...,Bastão Cera Impressora,Bastão Cera Impressora Cor Amarela Referência ...
3949,3949,435499,M,274968,S,N,7510,ARTIGOS PARA ESCRITÓRIO,NaN,NaN,75,UTENSÍLIOS DE ESCRITÓRIO E MATERIAL DE EXPEDIENTE,NaN,NaN,NaN,NaN,Material Papelão Cor Azul Tipo Com Abas Largur...,Pasta Arquivo,Pasta Arquivo Material Papelão Cor Azul Tipo C...
146695,146695,576369,M,252824,S,N,2990,"ACESSÓRIOS DIVERSOS DE MOTORES, EXCETO OS DE A...",NaN,NaN,29,ACESSÓRIOS DE MOTORES,NaN,NaN,NaN,NaN,"Uso Motor Aplicação Veiculo Gm, Mod. Corsa Hat...",Prato Assento Valvula,Prato Assento Valvula Uso Motor Aplicação Veic...
133799,133799,563735,M,410350,N,N,6640,EQUIPAMENTOS E ARTIGOS DE LABORATÓRIO,NaN,NaN,66,INSTRUMENTOS E EQUIPAMENTOS DE LABORATÓRIO,NaN,NaN,NaN,NaN,Graduação Graduado Característica Adicional Ap...,Microtubo,Microtubo Graduação Graduado Característica Ad...


In [9]:
queries_relv = df_catmar_relv['MATERIAL_TEXT'].to_list()
queries_relv

['Anticorpo Tipo Monoclonal De Camundongo Atividade Anti Cd45-Ro',
 'Ferramenta Padrão Furo Quadrado Passo 2x1 Tipo Pente Perfuração Uso Wire-O/Espiral',
 'Lona Plástica Cor Preta Largura 8MB Aplicação Cobertura E Proteção Espessura 150A',
 'Impressora Laser Tensão Alimentação 110/220V Resolução Impressão 600 X 600MM Capacidade Memórias 64W Capacidade Folha 600MBPS Velocidade Impressão Colorida 16X Velocidade Impressão Preto E Branco 16X Tipo Papel Carta, A4, A5, B5, Oficio, Executivo, Transparênci Características Adicionais Interface Paralela Padrão Centronics Bidirecional, Compatibilidade Protocolo Tcp/Ip',
 'Aparelho Ar Condicionado Tipo Split Capacidade Refrigeração 24.000KGF Modelo Split Inverter Tensão 127/220V Características Adicionais 1 Controle Remoto/Quente-Frio/Desumidificação/Filtro',
 'Tampa Condulete Material Pvc Cor Cinza Bitola 3/4POL Aplicação 2 Tomadas Universal',
 'Bastão Cera Impressora Cor Amarela Referência Impressora Tektronix 560 Durabilidade 10.000 Páginas Ref

In [13]:
import random
repeticao = 10

def create_queries(resp, queries):
    for j in range(repeticao):
        for query in queries:
            splited = query.split()
            for i in range(len(splited)):
                    if i >= 5:
                         break
                    resp.append({'query': ' '.join(splited[0: i+1]), 'querySize': i+1, 'repetition': j+1})

resp = []
create_queries(resp, queries_relv)

random.shuffle(resp)
time_queries = []
for q in resp:
    q_allmini = copy.deepcopy(q)
    q_allmini['abordagem'] = 'all_mini'
    time_queries.append(q_allmini)
    
    q_bv2 = copy.deepcopy(q)
    q_bv2['abordagem'] = 'q_bv2'
    time_queries.append(q_bv2)

    q_mpnet = copy.deepcopy(q)
    q_mpnet['abordagem'] = 'mpnet'
    time_queries.append(q_mpnet)
    
    q_labase = copy.deepcopy(q)
    q_labase['abordagem'] = 'labase'
    time_queries.append(q_labase)
df_test_relv = pd.DataFrame(time_queries)
df_test_relv
 

,query,querySize,repetition,abordagem
0,Reagente,1,8,all_mini
1,Reagente,1,8,q_bv2
2,Reagente,1,8,mpnet
3,Reagente,1,8,labase
4,Reagente Para Diagnóstico,3,9,all_mini
...,...,...,...,...
7995,Mepolizumabe,1,9,labase
7996,Microfone Tipo,2,10,all_mini
7997,Microfone Tipo,2,10,q_bv2
7998,Microfone Tipo,2,10,mpnet


In [14]:
df_test_relv[['querySize']].value_counts()

querySize
1            1600
2            1600
3            1600
4            1600
5            1600
Name: count, dtype: int64

In [44]:
df_test_relv.to_csv("ruidosCsvEmbeddings/sample_test_catmat_tempo_embeddings.csv")

In [16]:
df_test_relv = pd.read_csv("ruidosCsvEmbeddings/sample_test_catmat_tempo_embeddings.csv")
df_test_relv

,Unnamed: 0,query,querySize,repetition,abordagem
0,0,Reagente,1,8,all_mini
1,1,Reagente,1,8,q_bv2
2,2,Reagente,1,8,mpnet
3,3,Reagente,1,8,labase
4,4,Reagente Para Diagnóstico,3,9,all_mini
...,...,...,...,...,...
7995,7995,Mepolizumabe,1,9,labase
7996,7996,Microfone Tipo,2,10,all_mini
7997,7997,Microfone Tipo,2,10,q_bv2
7998,7998,Microfone Tipo,2,10,mpnet


In [43]:
df_test_relv = df_test_relv.loc[:, ~df_test_relv.columns.str.contains('^Unnamed')]


In [6]:
# es = Elasticsearch(
#     "http://150.165.75.163:9200",
#     basic_auth=("elastic", "GnlrL41DQapI3"),
#     request_timeout=999999999
# )

# es.info()
es = Elasticsearch(
    "http://150.165.75.163:9202",
    basic_auth=("elastic", "teste123"),
    request_timeout=999999999
)

es.info()


ObjectApiResponse({'name': 'es01', 'cluster_name': 'es-docker-cluster', 'cluster_uuid': '8pFe7X9oRge9WST4LcJmzw', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

['matlex', 'index_lex_nf', 'index_s_nf', 'mat_semaanc']

In [8]:
destino_consulta = 'matlex'
match_query = 'fone wireless'

query = {
        "query": {
            "match": {
                "material_text": match_query 
            }
        }
    }

response = es.search(index=destino_consulta, body=query)
for hit in response['hits']['hits']:
    print(hit['_source']['material_text']) 


Fone Ouvido Tipo Fone Para Estúdio Roland Rh-05
Fone Ouvido Comprimento Fio 1,80M Potência 100MS Impedância (Fone) 32 Ohms, (Microfone) 2,2 Kohms Freqüência (Fone) 20hz A 20khz, (Microfone) 50 A 20khz Tipo Fone Arco Características Adicionais Controle De Volume No Fio Do Fone, Microcomputador
Fone Ouvido Comprimento Fio 1,50MM Freqüência 225KVA Características Adicionais Fone Lateral Em Espuma
Fone Ouvido Tipo Headset Aplicação Mesa Telefônica Digital Freqüência 900V Tipo Fone Headset S/Fio Possuindo Fone C/Suporte P/Cabeça Ou Alcance 100MS
Fone Ouvido Tipo Fone Headphone Tipo Semi-Aberto Impedância 600MBPS Sensibilidade 88DBM Freqüência 15 A 20.000HZ
Cartão Rede Padrão Bluetooth Aplicação Cliente Wireless Tipo Protocolo Pci
Equipamento Wireless Frequência 2.4MHZ Padrão 802.11 A/G (Wi-Fi)
Fone Ouvido Impedância 32UN Freqüência 20AWG Tipo Fone Headphone Tipo Dinâmico Sensibilidade 96 Potência 20AWG
Fone Ouvido Tipo Profissional
Fone Ouvido Modelo Monoauricular Tipo Aparelho Telefônico C

In [4]:
from sentence_transformers import SentenceTransformer
modelL6V2 = SentenceTransformer('all-MiniLM-L6-v2')
modelBV2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
modelMult = SentenceTransformer('sentence-transformers/quora-distilbert-multilingual')
modelLASE = SentenceTransformer('sentence-transformers/LaBSE')

In [21]:
destino_consulta = 'mat_semaanc'
match_query = modelL6V2.encode('fone wireless')

query_smt= {
    'field': 'all_mini_base_vector',
    'query_vector': match_query.tolist(),
    'k': 10,
    'num_candidates': 50
}


response = es.search(index=destino_consulta, knn=query_smt)

for hit in response['hits']['hits']:
    print(hit['_source']['material_text'])

Fone Ouvido Comprimento Fio 1,80M Potência 100MS Impedância (Fone) 32 Ohms, (Microfone) 2,2 Kohms Freqüência (Fone) 20hz A 20khz, (Microfone) 50 A 20khz Tipo Fone Arco Características Adicionais Controle De Volume No Fio Do Fone, Microcomputador
Equipamento Wireless Frequência 2.4MHZ Padrão 802.11 A/G (Wi-Fi)
Equipamento Wireless Taxa Transmissão 300L Frequência 2.400 - 2.483,50MHZ Características Adicionais 3 Antenas Omni 3dbi,Access Point,Repetidor,Bridge Tipo Repetidora
Equipamento Wireless Padrão 802.11 A/B/G/N/Ac Aplicação Conexão Sem Fio De Equipamentos Em Rede Frequência 5  - 2,4GHZ
Antena Tipo Grade Características Adicionais Para Redes Wireless Em 5,8 Ghz Potência Máxima 100MS Freqüência 5725 A 5850MHZ Impedância 50CPS Ganho 27dbi Modelo Oiw-5827g
Fone Ouvido Aplicação Aparelho Telefônico Ip Tipo Fone Headset C/Controle Volume Microfone E Áudio;Stereo Freqüência (Microfone) 100hz-8khz;(Auto-Falante) 20hz-20KHZ Características Adicionais Compatível Com Windows; Auto-Falante Diâ

In [36]:
from requests.auth import HTTPBasicAuth

index_consulta = 'mat_semaanc'
match_query = modelL6V2.encode('fone wireless')
print(index_consulta)
print('----------')

script_query = {
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'bv2_base_vector') + 1.0",
                "params": {"query_vector": match_query.tolist()}
            }
        }
    }
    }
start_time = time.perf_counter()

response = requests.get('http://150.165.75.163:9202/' + index_consulta + '/_search', json=query, headers={'Content-type': 'application/json'},  auth=HTTPBasicAuth('elastic', 'teste123'))
time_elapsed = time.perf_counter() - start_time
total_time = time_elapsed
print(response)
print('--------------')
print(requests.codes.ok)
if response.status_code == requests.codes.ok:
    response_data = json.loads(response.content)
    print(response.content)
    hits = response_data['hits']['hits']
    
    modelos = [hit['_source']['material_text'] for hit in hits]
    print(modelos)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

mat_semaanc
----------
<Response [200]>
--------------
200
b'{"took":3,"timed_out":false,"_shards":{"total":3,"successful":3,"skipped":0,"failed":0},"hits":{"total":{"value":309,"relation":"eq"},"max_score":11.702267,"hits":[{"_index":"mat_semaanc","_id":"112050","_score":11.702267,"_source":{"id_material":546960,"codigo_classe":5965,"descricao_classe":"FONES, MICROFONES E ALTO-FALANTES","codigo_grupo":59,"caracteristica_busca":"Tipo Fone Para Est\xc3\xbadio Roland Rh-05","nome_material":"Fone Ouvido","material_text":"Fone Ouvido Tipo Fone Para Est\xc3\xbadio Roland Rh-05","all_mini_base_vector":[-0.042105939239263535,0.02117302268743515,-0.0032547791488468647,-0.06557699292898178,-0.03762303292751312,0.0791659951210022,-0.028765415772795677,0.10977226495742798,-0.0407315157353878,0.018514025956392288,0.07191120088100433,-0.011698110029101372,-0.08830641210079193,-0.04209760203957558,-0.06329841166734695,-0.03706518933176994,-0.0639345720410347,0.05503753945231438,0.036421433091163635,

In [2]:
import gc
import torch
import json
# experiment_relv_embeddings_temp1.log
import logging
# logging.getLogger('requests').setLevel(logging.DEBUG)
# logging.getLogger('urllib3').setLevel(logging.DEBUG)


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
from requests.auth import HTTPBasicAuth

logging.basicConfig(filename= 'experiment_relv_embeddings_temp_catmat.log',
    filemode='a',
    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S',
    level=logging.DEBUG)


def run(file, start=0, half=False):
    total_time = 0
    # processed_queries = 0
    experiment_queries = pd.read_csv(file)
    # experiment_queries['result'] = None

    num_rows = len(experiment_queries)
    mid_point = num_rows // 2

    if half:
        start = mid_point
        end = num_rows
    else:
        end = mid_point

    print("start: ", start)
    print("end: ", end)
    
    # if 'result' not in experiment_queries.columns:
    #     experiment_queries['result'] = None
    # if 'time' not in experiment_queries.columns:
    #     experiment_queries['time'] = None

    for key, value in experiment_queries.iloc[start:end].iterrows():    
        if(key % 800 == 0):
            print('key', key)
        logging.info(f'Iniciando processamento da query ({value["repetition"]}): {value["query"]})')
        try:

            match_query = ''
            query = ''          
            if(value['abordagem']) == 'all_mini':
                    index_consulta = 'mat_semaanc'
                    match_query = modelL6V2.encode(value['query'],show_progress_bar = False )
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'all_mini_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        }
                    }
            
            elif(value['abordagem']) == 'q_bv2':
                    index_consulta = 'mat_semaanc'
                    match_query = modelBV2.encode(value['query'], show_progress_bar = False)
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'bv2_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        }
                    }
     
            elif(value['abordagem']) == 'mpnet':
                    index_consulta = 'mat_semaanc'
                    match_query = modelMult.encode(value['query'],show_progress_bar = False)
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'mpnet_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        }
                    }

            elif(value['abordagem']) == 'labase':
                    index_consulta = 'mat_semaanc'
                    match_query = modelLASE.encode(value['query'],show_progress_bar = False)
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'labase_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        }
                    }    

            sucess = False
            while(not sucess):
                start_time = time.perf_counter()
                response = requests.get('http://150.165.75.163:9202/' + index_consulta + '/_search', json=query, headers={'Content-type': 'application/json'},  auth=HTTPBasicAuth('elastic', 'teste123'))
                
                time_elapsed = time.perf_counter() - start_time
                total_time += time_elapsed
                
                if response.status_code == requests.codes.ok:
                    sucess = True
                
            # processed_queries += 1
            # experiment_queries['result'][key] = response.content
            experiment_queries['time'][key] = time_elapsed

            # remaining_time = (total_time / processed_queries) * (len(experiment_queries) - processed_queries)

            logging.info(f'A query levou {time_elapsed:.6f}s')
            # logging.info(f'Faltam aproximadamente {int(remaining_time // 3600)}h, {int((remaining_time % 3600) // 60)}m e {math.floor(remaining_time % 60)}s')
        
        except Exception as error:
            logging.error(f'Erro na consulta: {error}')
    experiment_queries.to_csv(file)

In [26]:
file_name = '/mnt/DADOS/melqui/busca-smt/ruidosCsvEmbeddings/sample_test_catmat_tempo_embeddings.csv'
# df_test_relv['result'] = None
df_test_relv['time'] = None
df_test_relv.to_csv(file_name)


In [39]:
df_test_relv

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,query,querySize,repetition,abordagem,time
0,0,0,0,0,Reagente,1,8,all_mini,0.136637
1,1,1,1,1,Reagente,1,8,q_bv2,0.355502
2,2,2,2,2,Reagente,1,8,mpnet,0.351991
3,3,3,3,3,Reagente,1,8,labase,0.352558
4,4,4,4,4,Reagente Para Diagnóstico,3,9,all_mini,0.079033
...,...,...,...,...,...,...,...,...,...
7995,7995,7995,7995,7995,Mepolizumabe,1,9,labase,0.121064
7996,7996,7996,7996,7996,Microfone Tipo,2,10,all_mini,0.076536
7997,7997,7997,7997,7997,Microfone Tipo,2,10,q_bv2,0.139868
7998,7998,7998,7998,7998,Microfone Tipo,2,10,mpnet,0.140698


In [31]:
run(file_name)

start:  0
end:  4000
key 0
key 800
key 1600
key 2400
key 3200


In [32]:
run(file_name, half=True)

start:  4000
end:  8000
key 4000
key 4800
key 5600
key 6400
key 7200


In [45]:
df_test_relv = pd.read_csv("ruidosCsvEmbeddings/sample_test_catmat_tempo_embeddings.csv")
# df_test_relv['time']

In [46]:
df_test_relv

,Unnamed: 0,query,querySize,repetition,abordagem,time
0,0,Reagente,1,8,all_mini,0.136637
1,1,Reagente,1,8,q_bv2,0.355502
2,2,Reagente,1,8,mpnet,0.351991
3,3,Reagente,1,8,labase,0.352558
4,4,Reagente Para Diagnóstico,3,9,all_mini,0.079033
...,...,...,...,...,...,...
7995,7995,Mepolizumabe,1,9,labase,0.121064
7996,7996,Microfone Tipo,2,10,all_mini,0.076536
7997,7997,Microfone Tipo,2,10,q_bv2,0.139868
7998,7998,Microfone Tipo,2,10,mpnet,0.140698


In [49]:
result = pd.read_csv('ruidosCsvEmbeddings/sample_test_catmat_tempo_embeddings.csv')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8000 non-null   int64  
 1   query       8000 non-null   object 
 2   querySize   8000 non-null   int64  
 3   repetition  8000 non-null   int64  
 4   abordagem   8000 non-null   object 
 5   time        8000 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 375.1+ KB


In [50]:
result.describe()

,Unnamed: 0,querySize,repetition,time
count,8000.00000,8000.000000,8000.000000,8000.000000
mean,3999.50000,3.000000,5.500000,0.108432
std,2309.54541,1.414302,2.872461,0.023344
min,0.00000,1.000000,1.000000,0.065690
25%,1999.75000,2.000000,3.000000,0.110654
50%,3999.50000,3.000000,5.500000,0.113273
75%,5999.25000,4.000000,8.000000,0.121564
max,7999.00000,5.000000,10.000000,0.355502


In [51]:
result.groupby(['abordagem']).agg({'time': ['mean', 'std']})

time          
               mean       std
abordagem                    
all_mini   0.072476  0.006744
labase     0.120084  0.011816
mpnet      0.120275  0.011646
q_bv2      0.120893  0.011615

# Comparação  com léxico
Comparação do embedding de melhor Resultado com a abordagem Léxica

In [65]:
import random
repeticao = 10

def create_queries(resp, queries):
    for j in range(repeticao):
        for query in queries:
            splited = query.split()
            for i in range(len(splited)):
                    if i >= 5:
                         break
                    resp.append({'query': ' '.join(splited[0: i+1]), 'querySize': i+1, 'repetition': j+1})

resp = []
create_queries(resp, queries_relv)

random.shuffle(resp)
time_queries = []
for q in resp:
     q_semantic = copy.deepcopy(q)
     q_semantic['abordagem'] = 'semantico'
     time_queries.append(q_semantic)
     q_lexico = copy.deepcopy(q)
     q_lexico['abordagem'] = 'lexico'
     time_queries.append(q_lexico)

df_time_lx_smnt = pd.DataFrame(time_queries)
df_time_lx_smnt
 

,query,querySize,repetition,abordagem
0,Etiqueta Adesiva,2,7,semantico
1,Etiqueta Adesiva,2,7,lexico
2,Reagente Características Adicionais 1,4,1,semantico
3,Reagente Características Adicionais 1,4,1,lexico
4,Capa Características Adicionais Tipo,4,10,semantico
...,...,...,...,...
3995,Reagente Para Diagnóstico,3,5,lexico
3996,Reagente Para,2,1,semantico
3997,Reagente Para,2,1,lexico
3998,Conexão Hidráulica Tipo Fixação Soldável,5,9,semantico


In [10]:
df_time_lx_smnt[['querySize']].value_counts()

querySize
1            800
2            800
3            800
4            800
5            800
Name: count, dtype: int64

In [67]:
df_time_lx_smnt.to_csv("ruidosCsvEmbeddings/sample_test_catmat_tempo_lx_smnt.csv")

In [9]:
df_time_lx_smnt = pd.read_csv("ruidosCsvEmbeddings/sample_test_catmat_tempo_lx_smnt.csv")
df_time_lx_smnt

,Unnamed: 0.1,Unnamed: 0,query,querySize,repetition,abordagem,time
0,0,0,Etiqueta Adesiva,2,7,semantico,0.649557
1,1,1,Etiqueta Adesiva,2,7,lexico,0.002026
2,2,2,Reagente Características Adicionais 1,4,1,semantico,0.187597
3,3,3,Reagente Características Adicionais 1,4,1,lexico,0.002631
4,4,4,Capa Características Adicionais Tipo,4,10,semantico,0.179546
...,...,...,...,...,...,...,...
3995,3995,3995,Reagente Para Diagnóstico,3,5,lexico,0.002337
3996,3996,3996,Reagente Para,2,1,semantico,0.165359
3997,3997,3997,Reagente Para,2,1,lexico,0.002142
3998,3998,3998,Conexão Hidráulica Tipo Fixação Soldável,5,9,semantico,0.154886


In [11]:
from requests.auth import HTTPBasicAuth
logging.basicConfig(filename= 'experiment_relv_embeddings_temp_catmat_lx_smt.log',
    filemode='a',
    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S',
    level=logging.DEBUG)

def run_lx_smt(file):
    total_time = 0
    processed_queries = 0
    experiment_queries = pd.read_csv(file)
    # experiment_queries['result'] = None
    for key, value in experiment_queries.iterrows():
        if(key % 800 == 0):
            print('key', key)
        logging.info(f'Iniciando processamento da query ({value["repetition"]}): {value["query"]})')
        try:

            match_query = ''
            query = ''
            if(value['abordagem']) == 'semantico':
                index_consulta = 'index_s_nf'
                match_query = modelL6V2.encode(value['query'],show_progress_bar = False)
                
                query = {
                    "query": {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": "cosineSimilarity(params.query_vector, 'all_mini_base_vector') + 1.0",
                                "params": {"query_vector": match_query.tolist()}
                            }
                        }
                    }
                }
            
            else:
                index_consulta = 'index_lex_nf'
                match_query = value['query']
                
                query = {
                    "query": {
                        "match": {
                            "descricao_produto": match_query
                        }
                    }
                }    

            sucess = False
            while(not sucess):
                start_time = time.perf_counter()
                
                response = requests.get('http://150.165.75.163:9202/' + index_consulta + '/_search', json=query, headers={'Content-type': 'application/json'},  auth=HTTPBasicAuth('elastic', 'teste123'))
                time_elapsed = time.perf_counter() - start_time
                total_time += time_elapsed
                
                if response.status_code == requests.codes.ok:
                    sucess = True
            
            
            processed_queries += 1
            # experiment_queries['result'][key] = response.content
            experiment_queries['time'][key] = time_elapsed

            remaining_time = (total_time / processed_queries) * (len(experiment_queries) - processed_queries)

            logging.info(f'A query levou {time_elapsed:.6f}s')
            logging.info(f'Faltam aproximadamente {int(remaining_time // 3600)}h, {int((remaining_time % 3600) // 60)}m e {math.floor(remaining_time % 60)}s')

        except Exception as error:
            logging.error(f'Erro na consulta: {error}')
    experiment_queries.to_csv(file)

In [13]:
file_name = '/mnt/DADOS/melqui/busca-smt/ruidosCsvEmbeddings/sample_test_catmat_tempo_lx_smnt.csv'
# df_time_lx_smnt['result'] = None
df_time_lx_smnt['time'] = None
df_time_lx_smnt.to_csv(file_name)


In [14]:
df_time_lx_smnt

,Unnamed: 0.1,Unnamed: 0,query,querySize,repetition,abordagem,time
0,0,0,Etiqueta Adesiva,2,7,semantico,None
1,1,1,Etiqueta Adesiva,2,7,lexico,None
2,2,2,Reagente Características Adicionais 1,4,1,semantico,None
3,3,3,Reagente Características Adicionais 1,4,1,lexico,None
4,4,4,Capa Características Adicionais Tipo,4,10,semantico,None
...,...,...,...,...,...,...,...
3995,3995,3995,Reagente Para Diagnóstico,3,5,lexico,None
3996,3996,3996,Reagente Para,2,1,semantico,None
3997,3997,3997,Reagente Para,2,1,lexico,None
3998,3998,3998,Conexão Hidráulica Tipo Fixação Soldável,5,9,semantico,None


In [15]:
run_lx_smt(file_name)

key 0
key 800
key 1600
key 2400
key 3200


In [16]:
df_time_lx_smnt = pd.read_csv("ruidosCsvEmbeddings/sample_test_catmat_tempo_lx_smnt.csv")


In [20]:
df_time_lx_smnt

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,query,querySize,repetition,abordagem,time
0,0,0,0,0,Etiqueta Adesiva,2,7,semantico,0.182772
1,1,1,1,1,Etiqueta Adesiva,2,7,lexico,0.002138
2,2,2,2,2,Reagente Características Adicionais 1,4,1,semantico,0.174785
3,3,3,3,3,Reagente Características Adicionais 1,4,1,lexico,0.002597
4,4,4,4,4,Capa Características Adicionais Tipo,4,10,semantico,0.167701
...,...,...,...,...,...,...,...,...,...
3995,3995,3995,3995,3995,Reagente Para Diagnóstico,3,5,lexico,0.002770
3996,3996,3996,3996,3996,Reagente Para,2,1,semantico,0.188672
3997,3997,3997,3997,3997,Reagente Para,2,1,lexico,0.002610
3998,3998,3998,3998,3998,Conexão Hidráulica Tipo Fixação Soldável,5,9,semantico,0.187831


In [18]:
result_lx_smnt = pd.read_csv('ruidosCsvEmbeddings/sample_test_catmat_tempo_lx_smnt.csv')
result_lx_smnt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.3  4000 non-null   int64  
 1   Unnamed: 0.2  4000 non-null   int64  
 2   Unnamed: 0.1  4000 non-null   int64  
 3   Unnamed: 0    4000 non-null   int64  
 4   query         4000 non-null   object 
 5   querySize     4000 non-null   int64  
 6   repetition    4000 non-null   int64  
 7   abordagem     4000 non-null   object 
 8   time          4000 non-null   float64
dtypes: float64(1), int64(6), object(2)
memory usage: 281.4+ KB


In [19]:
result_lx_smnt.describe()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,querySize,repetition,time
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.00000,4000.000000
mean,1999.500000,1999.500000,1999.500000,1999.500000,3.00000,5.50000,0.087302
std,1154.844867,1154.844867,1154.844867,1154.844867,1.41439,2.87264,0.085564
min,0.000000,0.000000,0.000000,0.000000,1.00000,1.00000,0.001479
25%,999.750000,999.750000,999.750000,999.750000,2.00000,3.00000,0.002150
50%,1999.500000,1999.500000,1999.500000,1999.500000,3.00000,5.50000,0.103552
75%,2999.250000,2999.250000,2999.250000,2999.250000,4.00000,8.00000,0.168797
max,3999.000000,3999.000000,3999.000000,3999.000000,5.00000,10.00000,0.228347


In [21]:
result_lx_smnt.groupby(['abordagem']).agg({'time': ['mean', 'std']})

time          
               mean       std
abordagem                    
lexico     0.002329  0.001430
semantico  0.172276  0.013992